## Refactored code 

In [ ]:
import numpy as np

## Defining components of Phase I and Phase II 

In [ ]:
def initialize_phase1(A, b):
    """Constructs the Phase I problem by adding artificial variables."""
    m, n = A.shape
    A_ext = np.hstack((A, np.eye(m)))  # Add artificial variables
    c_ext = np.zeros(n + m)
    c_ext[n:] = 1  # Minimize sum of artificial variables
    initial_basis = list(range(n, n + m))  # Artificial vars as initial basis
    return A_ext, b, c_ext, initial_basis

def compute_reduced_costs(A, c, basis):
    """Computes reduced cost vector."""
    B = A[:, basis]
    B_inv = np.linalg.inv(B)
    c_B = c[basis]
    return c - np.dot(c_B.T @ B_inv, A)

def select_entering_variable(c_bar, basis):
    """Selects the entering variable (smallest index rule)."""
    nonbasic_indices = [j for j in range(len(c_bar)) if j not in basis]
    entering_candidates = [j for j in nonbasic_indices if c_bar[j] < 0]
    return min(entering_candidates) if entering_candidates else None

def compute_direction_vector(A, basis, entering_variable):
    """Computes the direction vector d."""
    B = A[:, basis]
    B_inv = np.linalg.inv(B)
    d = np.zeros(A.shape[1])
    d[basis] = -np.dot(B_inv, A[:, entering_variable])
    d[entering_variable] = 1
    return d

def select_leaving_variable(b, d, basis):
    """Determines the leaving variable using the minimum ratio rule."""
    step_sizes = [
        b[i] / -d[basis[i]] if d[basis[i]] < 0 else float('inf')
        for i in range(len(basis))
    ]
    leaving_index = np.argmin(step_sizes)
    return basis[leaving_index] if step_sizes[leaving_index] != float('inf') else None

def update_basis(basis, entering_variable, leaving_variable):
    """Updates the basis with the entering and leaving variables."""
    basis[basis.index(leaving_variable)] = entering_variable

## Simplex phase I and phase II

In [ ]:
def simplex_phase1(A, b):
    """Phase I: Finds an initial feasible basis."""
    A_ext, b, c_ext, initial_basis = initialize_phase1(A, b)
    basis = simplex_phase2(A_ext, b, c_ext, initial_basis, phase1=True)
    
    # If any artificial variable is in the final basis, LP is infeasible
    if basis is None or any(var >= A.shape[1] for var in basis):
        return None  # No feasible solution
    
    return basis  # Feasible basis for Phase II

def simplex_phase2(A, b, c, initial_basis, phase1=False):
    """Phase II: Solves the main optimization problem."""
    basis = initial_basis.copy()
    
    while True:
        c_bar = compute_reduced_costs(A, c, basis)
        entering_variable = select_entering_variable(c_bar, basis)
        
        if entering_variable is None:  # Optimal solution found
            basis.sort()
            return basis
        
        d = compute_direction_vector(A, basis, entering_variable)
        if np.all(d[basis] >= 0):  # Unbounded problem
            return None
        
        leaving_variable = select_leaving_variable(b, d, basis)
        if leaving_variable is None:  # No valid leaving variable
            return None
        
        update_basis(basis, entering_variable, leaving_variable)

## Sample problem and implementation

In [6]:
# Define the constraint matrix A, right-hand side b, and cost vector c
A = np.array([[1, 1, 1, 0], 
              [2, 3, 0, 1]])

b = np.array([4, 12])

c = np.array([-3, -1, 0, 0])  # Objective function coefficients

# Run Phase I to find an initial feasible basis
initial_basis = simplex_phase1(A, b)

if initial_basis:
    print("Initial feasible basis found:", initial_basis)
    
    # Run Phase II (actual optimization)
    optimal_basis = simplex_phase2(A, b, c, initial_basis)
    print("Optimal basis after Phase II:", optimal_basis)
else:
    print("The original LP is infeasible.")

Initial feasible basis found: [1, 3]
Optimal basis after Phase II: [0, 3]
